In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numba as nb
from math import exp
from tqdm.auto import tqdm

In [2]:
# %pip install ipywidgets

In [3]:
# %pip install numba

In [4]:
%matplotlib inline

In [5]:
k_0 = 1
a_1 = 0.0134
a_2 = 2.049
b_1 = 1
b_2 = 0.563 * 1e-3
c_1 = 4.35 * 1e-4
c_2 = 0.528 * 1e+5
m_1 = 1
m_2 = 1
alpha_0 = 0.05
alpha_n = 0.01
l = 10
T_0 = 300
r = 0.5
f_max = 50
t_max = 60
ac = alpha_0 * alpha_n * l / (alpha_0 - alpha_n)
ad = alpha_n * l / (alpha_n - alpha_0)

h = 1e-3
tau = 2e-2

In [6]:
@nb.jit
def lmbda(T):
    return a_1 * (b_1 + c_1 * T ** m_1)

@nb.jit
def c(T, a_2):
    return a_2 + b_2 * T ** m_2 - c_2 / T / T

@nb.jit
def k(T):
    return k_0 * T * T / 90000

@nb.jit
def alpha(x):
    return ac / (x - ad)

@nb.jit
def p(x):
    return 2 / r * alpha(x)

@nb.jit
def F0(t):
    return f_max / t_max * t * exp(-(t / t_max - 1))

@nb.jit
def F0_const(t):
    return f_max / t_max

@nb.jit
def F0_freq(t, freq):
    period = 1 / freq
    if abs(t / period - int(t / period)) < 1e-4:
        return f_max / t_max * t * np.exp(-(t / t_max - 1))
    else:
        return 0

AttributeError: module 'numba' has no attribute 'jit'

In [ ]:
@nb.jit
def f(x, T, t, f_freq):
    if f_freq == 0:
        return k(T) * F0(t) * exp(-k(T) * x) + 2 * T_0 / r * alpha(x)
    return k(T) * F0_freq(t, f_freq) * exp(-k(T) * x) + 2 * T_0 / r * alpha(x)

@nb.jit
def kappa(z_this, z_next):
    return 2 * lmbda(z_this) * lmbda(z_next) / (lmbda(z_this) + lmbda(z_next))

def stop_condition(z_relax_iter, z_next_iter):
    eps = 1e-2
    max_err = np.max(np.abs((z_next_iter - z_relax_iter) / z_next_iter))

    return max_err < eps

In [ ]:
@nb.jit
def tridiagonal(x_s, prev, cur_layer, cur_time, a_2, f_freq):
    half_z = (prev[0] + prev[1]) / 2
    half_last = (prev[-1] + prev[-2]) / 2
    prev_half = (cur_layer[0] + cur_layer[1]) / 2
    prev_last_half = (cur_layer[-1] + cur_layer[-2]) / 2
    
    h = x_s[1] - x_s[0]
    K_0 = h / 8 * c(half_z, a_2) + h / 4 * c(prev[0], a_2) + kappa(prev[0], prev[1]) * tau / h + alpha_0 * tau + h * tau / 4 * p(0) + h * tau / 8 * p(h / 2)
    M_0 = h / 8 * c(half_z, a_2) - kappa(prev[0], prev[1]) / h * tau + h * tau / 8 * p(h / 2)
    P_0 = h / 4 * (prev_half * c(half_z, a_2) + cur_layer[0] * c(prev[0], a_2)) + alpha_0 * T_0 * tau + h * tau / 4 * (f(0, prev[0], cur_time, f_freq) + f(h/2, half_z, cur_time, f_freq))

    xi = np.zeros(x_s.shape)
    etha = np.zeros(x_s.shape)
    xi[1] = -M_0 / K_0
    etha[1] = P_0 / K_0
    
    for i in range(1, len(prev) - 1):
        a_n = kappa(prev[i - 1], prev[i]) * tau / (x_s[i] - x_s[i - 1])
        c_n = kappa(prev[i], prev[i + 1]) * tau / (x_s[i + 1] - x_s[i])
        h = (x_s[i + 1] - x_s[i])
        b_n = a_n + c_n + c(prev[i], a_2) * h + p(h * i) * tau * h
        f_n = c(prev[i], a_2) * cur_layer[i] * h + f(h * i, prev[i], cur_time, f_freq) * tau * h

        xi[i+1] = c_n / (b_n - a_n * xi[i])
        etha[i+1] = (f_n + a_n * etha[i])/(b_n - a_n * xi[i])
    
    z = np.zeros(x_s.shape)

    K_N = h / 8 * c(half_last, a_2) - kappa(prev[-2], prev[-1]) * tau / h + tau * h / 8 * p(l - h / 2)
    M_N = h / 4 * c(prev[-1], a_2) + h / 8 * c(half_last, a_2) + tau / h * kappa(prev[-2], prev[-1]) + tau * alpha_n + tau * h / 4 * p(l) + tau * h / 8 * p(l - h / 2)
    P_N = alpha_n * T_0 * tau + tau * h / 4 * (f(l, prev[-1], cur_time, f_freq) + f(l - h/2, half_last, cur_time, f_freq)) + h / 4 * (c(prev[-1], a_2) * cur_layer[-1] + c(half_last, a_2) * prev_last_half)

    z[-1] = (P_N - M_N * etha[-1]) / (K_N + M_N * xi[-1])

    for i in range(len(z) - 2, -1, -1):
        z[i] = z[i+1] * xi[i+1] + etha[i+1]

    return z

In [ ]:
def get_next_layer(x_s, cur_layer, cur_time, a_2, f_freq):
    xi = 0.5
    next_iter = cur_layer
    relax_iter = cur_layer

    while True:
        next_iter = tridiagonal(x_s, relax_iter, cur_layer, cur_time, a_2, f_freq)

        if stop_condition(relax_iter, next_iter):
            break

        relax_iter += xi * (next_iter - relax_iter)

    return next_iter

In [ ]:
def solve(x_s, a_2, f_freq = 0):
    t_s = []
    t = 0
    while t <= t_max * 3:
        t_s.append(t)
        t += tau
    t_s.append(t)
#     t_s = [i * tau for i in range(int(t_max * 3 / tau))]
#     t_s.append(t_max * 3)
    t_s = np.array(t_s)
    
    
    res = [
        [T_0 for j in range(len(x_s))],
    ]
    
    res = [
        np.ones(x_s.shape) * T_0
    ]

    for i in tqdm(range(len(t_s)-1)):
        layer = get_next_layer(x_s, res[i], t_s[i + 1], a_2, f_freq)
        res.append(layer)

    return res, t_s

In [ ]:
x = np.linspace(0, 1, int(l/h) + 1)]
x = x / np.max(x) * l

T, t = solve(x, a_2)

In [ ]:
M = len(t)
view = slice(int(len(x)))
T_for_graph = [T[0], T[M // 4], T[M // 2], T[3 * M // 4], T[-1]]
plt.plot(x[view], T_for_graph[0][view], '#00f', label='t = 0')
plt.plot(x[view], T_for_graph[1][view], '#40c', label=f't = {t_max//4}')
plt.plot(x[view], T_for_graph[2][view], '#808', label=f't = {t_max // 2}')
plt.plot(x[view], T_for_graph[3][view], '#c04', label=f't = {3 * t_max // 4}')
plt.plot(x[view], T_for_graph[4][view], '#f00', label=f't = {t_max}')
plt.legend()

plt.grid()

In [ ]:
M = len(t)
T_for_graph = [T[0], T[M // 4], T[M // 2], T[3 * M // 4], T[-1]]
df = pd.DataFrame({
    'x': x,
    't = 0': T_for_graph[0],
    f't = {t_max//4}': T_for_graph[1],
    f't = {t_max // 2}': T_for_graph[2],
    f't = {3 * t_max // 4}': T_for_graph[3],
    f't = {t_max}': T_for_graph[4],
})
df

In [ ]:
T_c = [[T[i][0] for i in range(len(T))]]
for a_2 in (1.549, 2.549, 3.049):
    T_this, _, = solve(x, a_2)
    T_c.append([T_this[i][0] for i in range(len(T_this))])

plt.plot(t, T_c[1], 'b', label='a2 = 1.549')
plt.plot(t, T_c[0], 'r', label='a2 = 2.049')
plt.plot(t, T_c[2], 'g', label='a2 = 2.549')
plt.plot(t, T_c[3], 'm', label='a2 = 3.049')
plt.legend()

plt.grid()

In [ ]:
T_f = []
a_2 = 2.049

for freq in (1, 2, 5, 10, 20):
    T_this, _ = solve(x, a_2, freq)
    T_f.append([T_this[i][0] for i in range(len(T_this) // 2)])

t = t[:len(T_f[0])]

plt.plot(t, T_f[0], 'r', label='v = 1')
plt.plot(t, T_f[1], 'b', label='v = 2')
plt.plot(t, T_f[2], 'g', label='v = 5')
plt.plot(t, T_f[3], 'm', label='v = 10')
plt.plot(t, T_f[4], 'k', label='v = 20')
plt.legend()

plt.grid()